# Simulación de la cadena de Markov en el procedimiento RACH

## 1. Introducción

En sistemas LTE/5G, los dispositivos (**User Equipment, UE**) deben acceder a la red mediante un procedimiento de acceso aleatorio llamado **Random Access Channel (RACH)**.  
Cada UE dispone de hasta tres intentos para obtener respuesta del eNB, y cada intento puede resultar en **éxito** o **colisión** (otro UE eligió el mismo preámbulo).  

El objetivo de esta práctica es:

- Modelar el proceso de acceso usando **cadenas de Markov**.
- Simular el proceso en Python para millones de UEs.
- Comparar resultados simulados con valores teóricos analíticos.
- Analizar la probabilidad de éxito, el número promedio de intentos y la distribución de estados.  

Las **cadenas de Markov** son modelos matemáticos de procesos aleatorios donde el siguiente estado depende únicamente del estado actual (propiedad de memoria cero). En este caso, cada intento del UE es un estado transitorio y el éxito o fracaso son estados absorbentes.



## 2. Modelo de la cadena de Markov

### Estados

| Estado | Descripción |
|--------|------------|
| S₀ | 1er intento |
| S₁ | 2do intento |
| S₂ | 3er intento |
| Éxito | UE logró acceso (absorción) |
| Fracaso | UE falló los 3 intentos (absorción) |

### Probabilidades de transición

- Probabilidad de éxito por intento: `p = 0.6`  
- Probabilidad de colisión por intento: `q = 0.4`  

Matriz de transición \(P\) (orden de estados: S₀, S₁, S₂, Éxito, Fracaso):

\[
P = 
\begin{bmatrix}
0 & 0.4 & 0 & 0.6 & 0 \\
0 & 0 & 0.4 & 0.6 & 0 \\
0 & 0 & 0 & 0.6 & 0.4 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
\]

- Estados S₀, S₁, S₂ → transitorios  
- Estados Éxito y Fracaso → absorbentes



## 3. Cálculos analíticos

Según la teoría de cadenas de Markov absorbentes:

- Submatriz de estados transitorios **Q** (3×3):  
  $$
  Q =
  \begin{bmatrix}
  0 & 0.4 & 0 \\
  0 & 0 & 0.4 \\
  0 & 0 & 0
  \end{bmatrix}
  $$
  
- Submatriz de transiciones a estados absorbentes **R** (3×2):  
  $$
  R =
  \begin{bmatrix}
  0.6 & 0 \\
  0.6 & 0 \\
  0.6 & 0.4
  \end{bmatrix}
  $$

- Matriz fundamental:  
  $$
  N = (I - Q)^{-1}
  $$

- Probabilidades de absorción:  
  $$
  B = N \cdot R
  $$

- Número esperado de intentos:  
  $$
  E[I] = \text{suma de la fila correspondiente a } S_0 \text{ en } N
  $$

### Resultados teóricos

- Probabilidad de éxito:  
  $$
  P(\text{éxito}) = 1 - q^3 = 1 - 0.4^3 = 0.936
  $$

- Probabilidad de fracaso:  
  $$
  P(\text{fracaso}) = q^3 = 0.064
  $$

- Número promedio de intentos:  
  $$
  E[I] = 1.56
  $$

  ![Grafica](GraficoProbAcumulada.png)

## 4. Resultado de Simulación

- Probabilidad de éxito: 0.935974
- Intentos promedio: 1.559434
- Distribución estacionaria aproximada:  
  - Éxito: 0.936 
  - Fracaso: 0.064

## 5. Simulación en Python 



In [1]:
import numpy as np

# Parámetros
p_exito = 0.6
p_fallo = 0.4
n_ue = 10**6

# Resultados
exitos = 0
intentos_totales = 0
estado_final = []

for _ in range(n_ue):
    intento = 0
    exito = False
    
    while intento < 3:
        intento += 1
        if np.random.rand() < p_exito:
            exitos += 1
            exito = True
            break
    intentos_totales += intento
    estado_final.append("Éxito" if exito else "Fracaso")

# a) Probabilidad de éxito
p_exito_sim = exitos / n_ue

# b) Número medio de intentos
prom_intentos = intentos_totales / n_ue

# c) Distribución estacionaria (aproximada)
from collections import Counter
counts = Counter(estado_final)
p_exito_stat = counts["Éxito"] / n_ue
p_fracaso_stat = counts["Fracaso"] / n_ue

print("Probabilidad de éxito:", p_exito_sim)
print("Intentos promedio:", prom_intentos)
print("Distribución estacionaria aproximada:")
print(f"Éxito: {p_exito_stat:.3f}, Fracaso: {p_fracaso_stat:.3f}")


Probabilidad de éxito: 0.935633
Intentos promedio: 1.560431
Distribución estacionaria aproximada:
Éxito: 0.936, Fracaso: 0.064
